In [1]:
import numpy as np
import pandas as pd
import folium

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [2]:
path = 'C:/Users/Nancy Sorroza Rojas/Desktop/Pool/Proyectos Data Science/Space X practice/data/spacex_launch_geo.csv'
spacex_df=pd.read_csv(path)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


# Task 1: Mark all launch sites on a map

In [3]:
# Create a Map center in United States
UUEE = folium.Map(
    location = [37.0902, -95.7129], 
    zoom_start=4, 
    tiles='Cartodb dark_matter')



# Instantiate a feature group for the lunch Sites in the dataframe
lunch_sites = folium.map.FeatureGroup()

# loop through the Latitud and Longitud of each Lunch Site
for lat, lng, in zip(spacex_df.Lat, spacex_df.Long):
    lunch_sites.add_child(
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='white',
            fill=True,
            fill_color='red',
            fill_opacity=0.5
        )
    )

# add lunch Sites to map
UUEE.add_child(lunch_sites)

# display map
UUEE

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#000080', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000080;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for lat, lng, l_site, in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=1000, color='#000080', fill=True).add_child(folium.Popup(f'{l_site}'))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#000080;"><b>%s</b></div>' % f'{l_site}',
            )
        )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# # Task 2: Mark the success/failed launches for each site on the map


In [8]:
map_sf = folium.Map(location=nasa_coordinate, zoom_start=4)

for lat, lng, l_site, in zip(spacex_df.Lat, spacex_df.Long, spacex_df['class']):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=10, color='#000080', fill=True).add_child(folium.Popup(f'{l_site}'))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(10,10),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#000080;"><b>%s</b></div>' % f'{l_site}',
            )
        )
    
    map_sf.add_child(circle)
    map_sf.add_child(marker)

map_sf

Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [9]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [10]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

spacex_df['marker_color'] = np.where(spacex_df['class']==1, 'green', 'red')

In [11]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    if record['class']==1:
        marker = folium.Marker(
            location=[record['Lat'], record['Long']],
            popup="launch was successed",
            tooltip="Successed",
            icon=folium.Icon(color=record['marker_color'], icon="info-sign")
        )
    if record['class']==0:
        marker = folium.Marker(
            location=[record['Lat'], record['Long']],
            popup="launch was failed",
            tooltip="Failed",
            icon=folium.Icon(color=record['marker_color'], icon="info-sign")
        )
    
    marker_cluster.add_child(marker)

site_map

# TASK 3: Calculate the distances between a launch site to its proximities

In [12]:
# !pip install geopy
from geopy.distance import geodesic

df_proximity = spacex_df.copy()

for i in range(len(launch_sites_df)):
    site_name = launch_sites_df.iloc[i]['Launch Site']
    site_coord = (
        launch_sites_df.iloc[i]['Lat'],
        launch_sites_df.iloc[i]['Long']
    )

    # crear la nueva columna vacía
    df_proximity[f'distance_km_{site_name}'] = df_proximity.apply(
        lambda row: geodesic(
            site_coord, 
            (row['Lat'], row['Long'])
        ).km,
        axis=1
    )

df_proximity

,Launch Site,Lat,Long,class,marker_color,distance_km_CCAFS LC-40,distance_km_CCAFS SLC-40,distance_km_KSC LC-39A,distance_km_VAFB SLC-4E
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
5,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
6,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
7,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
8,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645
9,CCAFS LC-40,28.562302,-80.577356,0,red,0.000000,0.112244,6.911215,3833.06645


Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
coastline_lat= 28.56339 
coastline_lon= -80.56782
coastline_coord = (coastline_lat, coastline_lon)

# Launch Site for 'CCAFS LC-40'
launch_site_lat=launch_sites_df.iloc[1]['Lat']
launch_site_lon=launch_sites_df.iloc[1]['Long']

CCAFS_LC_40_coord = (launch_site_lat, launch_site_lon)

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
   coastline_coord,
   icon=DivIcon(
       icon_size=(10,10),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#00cc44;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)
site_map

# TODO: Draw a PolyLine between a launch site to the selected coastline point

In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [CCAFS_LC_40_coord, coastline_coord]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [18]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [19]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
